In [1]:
import tensorflow as tf
import pandas as pd
import tensorflow.keras.backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras import regularizers
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.metrics import AUC
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras.applications.inception_v3 import preprocess_input
import cv2
import os
import random
import collections
from collections import defaultdict
from shutil import copy
from shutil import copytree, rmtree
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as img

In [2]:
#data loading function
def get_data_extract():
    if "food_data" in os.listdir():
        print("Dataset already exists")
    else:
        tf.keras.utils.get_file(
        './food-101.tar.gz',
        'http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz',
        cache_subdir='/content',
        extract=True,
        archive_format='tar',
        cache_dir=None)
        print("Dataset downloaded and extracted!")

In [3]:
#getting data
get_data_extract()

Dataset already exists


In [4]:
#train/test data extraction function
def prepare_data(filepath, src, dest):
    classes_images = defaultdict(list)
    with open(filepath, 'r') as txt:
        paths = [read.strip() for read in txt.readlines()]
        for p in paths:
            food = p.split('/')
            classes_images[food[0]].append(food[1] + '.jpg')

    for food in classes_images.keys():
        print("\nCopying images into ",food)
        if not os.path.exists(os.path.join(dest,food)):
            os.makedirs(os.path.join(dest,food))
        for i in classes_images[food]:
            copy(os.path.join(src,food,i), os.path.join(dest,food,i))
    print("Copying Done!")

In [5]:
#getting train data
prepare_data('food_data/food-101/meta/train.txt', 'food_data/food-101/images', 'food_data/food-101/train')


Copying images into  apple_pie

Copying images into  baby_back_ribs

Copying images into  baklava

Copying images into  beef_carpaccio

Copying images into  beef_tartare

Copying images into  beet_salad

Copying images into  beignets

Copying images into  bibimbap

Copying images into  bread_pudding

Copying images into  breakfast_burrito

Copying images into  bruschetta

Copying images into  caesar_salad

Copying images into  cannoli

Copying images into  caprese_salad

Copying images into  carrot_cake

Copying images into  ceviche

Copying images into  cheesecake

Copying images into  cheese_plate

Copying images into  chicken_curry

Copying images into  chicken_quesadilla

Copying images into  chicken_wings

Copying images into  chocolate_cake

Copying images into  chocolate_mousse

Copying images into  churros

Copying images into  clam_chowder

Copying images into  club_sandwich

Copying images into  crab_cakes

Copying images into  creme_brulee

Copying images into  croque_madam

In [6]:
#getting test data
prepare_data('food_data/food-101/meta/test.txt', 'food_data/food-101/images', 'food_data/food-101/test')


Copying images into  apple_pie

Copying images into  baby_back_ribs

Copying images into  baklava

Copying images into  beef_carpaccio

Copying images into  beef_tartare

Copying images into  beet_salad

Copying images into  beignets

Copying images into  bibimbap

Copying images into  bread_pudding

Copying images into  breakfast_burrito

Copying images into  bruschetta

Copying images into  caesar_salad

Copying images into  cannoli

Copying images into  caprese_salad

Copying images into  carrot_cake

Copying images into  ceviche

Copying images into  cheesecake

Copying images into  cheese_plate

Copying images into  chicken_curry

Copying images into  chicken_quesadilla

Copying images into  chicken_wings

Copying images into  chocolate_cake

Copying images into  chocolate_mousse

Copying images into  churros

Copying images into  clam_chowder

Copying images into  club_sandwich

Copying images into  crab_cakes

Copying images into  creme_brulee

Copying images into  croque_madam

In [7]:
#checking amount of images in train folders
train_files = sum([len(files) for i, j, files in os.walk("food_data/food-101/train")])
print("Total number of samples in train folder")
print(train_files)

Total number of samples in train folder
75750


In [8]:
#checking amount of images in test folders
test_files = sum([len(files) for i, j, files in os.walk("food_data/food-101/test")])
print("Total number of samples in test folder")
print(test_files)

Total number of samples in test folder
25250


In [9]:
#total amount of classes (different foods images)
foods_sorted = sorted(os.listdir('food_data/food-101/images'))
foods_sorted

['apple_pie',
 'baby_back_ribs',
 'baklava',
 'beef_carpaccio',
 'beef_tartare',
 'beet_salad',
 'beignets',
 'bibimbap',
 'bread_pudding',
 'breakfast_burrito',
 'bruschetta',
 'caesar_salad',
 'cannoli',
 'caprese_salad',
 'carrot_cake',
 'ceviche',
 'cheese_plate',
 'cheesecake',
 'chicken_curry',
 'chicken_quesadilla',
 'chicken_wings',
 'chocolate_cake',
 'chocolate_mousse',
 'churros',
 'clam_chowder',
 'club_sandwich',
 'crab_cakes',
 'creme_brulee',
 'croque_madame',
 'cup_cakes',
 'deviled_eggs',
 'donuts',
 'dumplings',
 'edamame',
 'eggs_benedict',
 'escargots',
 'falafel',
 'filet_mignon',
 'fish_and_chips',
 'foie_gras',
 'french_fries',
 'french_onion_soup',
 'french_toast',
 'fried_calamari',
 'fried_rice',
 'frozen_yogurt',
 'garlic_bread',
 'gnocchi',
 'greek_salad',
 'grilled_cheese_sandwich',
 'grilled_salmon',
 'guacamole',
 'gyoza',
 'hamburger',
 'hot_and_sour_soup',
 'hot_dog',
 'huevos_rancheros',
 'hummus',
 'ice_cream',
 'lasagna',
 'lobster_bisque',
 'lobster

In [10]:
#creating miniature dataset for quick testing different model architectures
def dataset_gen(food_list, src, dest):
    if os.path.exists(dest):
        rmtree(dest)
    os.makedirs(dest)
    for food_item in food_list :
        print("Copying images into",food_item)
        copytree(os.path.join(src,food_item), os.path.join(dest,food_item))

In [11]:
#preparing parameters for dataset_mini function to cteate mini datasets
food_list = ['hot_dog','ravioli','tuna_tartare']
src_train = 'food_data/food-101/train'
dest_train = 'food_data/food-101/train_mini'
src_test = 'food_data/food-101/test'
dest_test = 'food_data/food-101/test_mini'

In [12]:
#creating mini train set
dataset_gen(food_list, src_train, dest_train)

Copying images into hot_dog
Copying images into ravioli
Copying images into tuna_tartare


In [13]:
#creating mini train set
dataset_gen(food_list, src_test, dest_test)

Copying images into hot_dog
Copying images into ravioli
Copying images into tuna_tartare


In [14]:
#checking amount of images in mini train folders
print("Total number of samples in train folder")
train_files = sum([len(files) for i, j, files in os.walk(dest_train)])
print(train_files)

Total number of samples in train folder
2250


In [15]:
#checking amount of images in mini test folders
print("Total number of samples in test folder")
train_files = sum([len(files) for i, j, files in os.walk(dest_test)])
print(train_files)

Total number of samples in test folder
750


In [16]:
img_width, img_height = 299, 299
train_data_dir = 'food_data/food-101/train_mini'
validation_data_dir = 'food_data/food-101/test_mini'
batch_size = 64

In [17]:
train_datagen = ImageDataGenerator(
    preprocessing_function = preprocess_input,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(preprocessing_function = preprocess_input)

In [18]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

Found 2250 images belonging to 3 classes.


In [19]:
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

Found 750 images belonging to 3 classes.


In [20]:
#f1 metric
def f1(y_true, y_pred): 
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    
    return f1_val

In [21]:
def CNN(width, height, depth, n_classes, regularization, activization, initializer = 'he_normal'):
    
    if K.image_data_format() == "channels_first":
        input_shape = (depth, height, width)
        chan_dim = 1
    else:
        input_shape = (height, width, depth)
        chan_dim = -1
    
    model = Sequential(
        [
            layers.Conv2D(filters = 16, kernel_size = (7, 7), padding = 'valid', kernel_initializer = initializer,
                        strides = (2, 2), kernel_regularizer = regularization, input_shape = input_shape),
            
            layers.Conv2D(filters = 32, kernel_size = (3, 3), padding = 'same', activation = activization,
                        kernel_regularizer = regularization, kernel_initializer = initializer),
            layers.BatchNormalization(axis = chan_dim),
            layers.Conv2D(filters = 32, kernel_size = (3, 3), strides = (2, 2), padding = 'same', activation = activization,
                        kernel_regularizer = regularization, kernel_initializer = initializer),
            layers.BatchNormalization(axis = chan_dim),
            layers.Dropout(0.2),
            
            layers.Conv2D(filters = 64, kernel_size = (3, 3), padding = 'same', activation = activization,
                        kernel_regularizer = regularization, kernel_initializer = initializer),
            layers.BatchNormalization(axis = chan_dim),
            layers.Conv2D(filters = 64, kernel_size = (3, 3), strides = (2, 2), padding = 'same', activation = activization,
                        kernel_regularizer = regularization, kernel_initializer = initializer),
            layers.BatchNormalization(axis = chan_dim),
            layers.Dropout(0.2),
            
            layers.Conv2D(filters = 128, kernel_size = (3, 3), padding = 'same', activation = activization,
                        kernel_regularizer = regularization, kernel_initializer = initializer),
            layers.BatchNormalization(axis = chan_dim),
            layers.Conv2D(filters = 128, kernel_size = (3, 3), strides = (2, 2), padding = 'same', activation = activization,
                        kernel_regularizer = regularization, kernel_initializer = initializer),
            layers.BatchNormalization(axis = chan_dim),
            layers.Dropout(0.2),
                     
            layers.Flatten(),
            layers.Dense(512, kernel_initializer = initializer, activation = activization),
            layers.BatchNormalization(),
            layers.Dropout(0.5),
            
            layers.Dense(n_classes, activation = 'softmax'),

        ]
    )
    
    return model 

In [22]:
def optimizer_comparison(optimizer1, optimizer2, model_for_comp1, n_epoch):
    
    model_for_comp2 = tf.keras.models.clone_model(model_for_comp1)
    comparison_results = pd.DataFrame(index = ['Train loss', 'Train accuracy', 'Train f1', 'Train AUC', 'Validation loss', 'Validation accuracy', 'Validation f1', 'Validation AUC'])
    metrics_list = ['accuracy', f1, AUC(name = 'AUC')]
        
    model_for_comp1.compile(optimizer = optimizer1, loss = 'categorical_crossentropy', metrics = metrics_list)
    model_for_comp2.compile(optimizer = optimizer2, loss = 'categorical_crossentropy', metrics = metrics_list)
        
    history1 = model_for_comp1.fit(train_generator, validation_data = validation_generator, epochs = n_epoch, verbose = 0)
    history2 = model_for_comp2.fit(train_generator, validation_data = validation_generator, epochs = n_epoch, verbose = 0)
    
    comparison_results['First optimizer'] = [history1.history['loss'][-1], history1.history['accuracy'][-1], history1.history['f1'][-1],
                                            history1.history['AUC'][-1], history1.history['val_loss'][-1], history1.history['val_accuracy'][-1], 
                                            history1.history['val_f1'][-1], history1.history['val_AUC'][-1]]
    
    comparison_results['Second optimizer'] = [history1.history['loss'][-1], history1.history['accuracy'][-1], history1.history['f1'][-1],
                                            history1.history['AUC'][-1], history1.history['val_loss'][-1], history1.history['val_accuracy'][-1], 
                                            history1.history['val_f1'][-1], history1.history['val_AUC'][-1]]

    comparison_results.to_csv('Optimizer_comparison_results')
    
    return comparison_results

In [23]:
def activation_comparison(activation1, activation2, n_epoch):
    
    model_for_comp1 = CNN(299, 299, 3, 3, l2(0.00005), activation1)
    model_for_comp2 = CNN(299, 299, 3, 3, l2(0.00005), activation2)
        
    comparison_results = pd.DataFrame(index = ['Train loss', 'Train accuracy', 'Train f1', 'Train AUC', 'Validation loss', 'Validation accuracy', 'Validation f1', 'Validation AUC'])
    metrics_list = ['accuracy', f1, AUC(name = 'AUC')]
    
    model_for_comp1.compile(optimizer = Adam(learning_rate=0.001), loss = 'categorical_crossentropy', metrics = metrics_list)
    model_for_comp2.compile(optimizer = Adam(learning_rate=0.001), loss = 'categorical_crossentropy', metrics = metrics_list)
    
    history1 = model_for_comp1.fit(train_generator, validation_data = validation_generator, epochs = n_epoch, verbose = 0)
    history2 = model_for_comp2.fit(train_generator, validation_data = validation_generator, epochs = n_epoch, verbose = 0)
    
    comparison_results['First activation'] = [history1.history['loss'][-1], history1.history['accuracy'][-1], history1.history['f1'][-1],
                                            history1.history['AUC'][-1], history1.history['val_loss'][-1], history1.history['val_accuracy'][-1], 
                                            history1.history['val_f1'][-1], history1.history['val_AUC'][-1]]
    
    comparison_results['Second activation'] = [history1.history['loss'][-1], history1.history['accuracy'][-1], history1.history['f1'][-1],
                                            history1.history['AUC'][-1], history1.history['val_loss'][-1], history1.history['val_accuracy'][-1], 
                                            history1.history['val_f1'][-1], history1.history['val_AUC'][-1]]

    comparison_results.to_csv('Activation_comparison_results')
    
    return comparison_results

In [24]:
#RELU vs tanh
activation_comparison = pd.read_csv('Activation_comparison_results')

In [25]:
activation_comparison

,Unnamed: 0,First activation,Second activation
0,Train loss,0.608128,0.608128
1,Train accuracy,0.774667,0.774667
2,Train f1,0.771333,0.771333
3,Train AUC,0.915871,0.915871
4,Validation loss,0.846285,0.846285
5,Validation accuracy,0.716000,0.716000
6,Validation f1,0.709009,0.709009
7,Validation AUC,0.870251,0.870251


In [26]:
#Adam 0.001 learning rate vs SGD 0.0001 learning rate 0.9 momentum
optimizer_comparison = pd.read_csv('Optimizer_comparison_results')

In [27]:
optimizer_comparison

,Unnamed: 0,First optimizer,Second optimizer
0,Train loss,0.726076,0.726076
1,Train accuracy,0.714222,0.714222
2,Train f1,0.698641,0.698641
3,Train AUC,0.877116,0.877116
4,Validation loss,0.904731,0.904731
5,Validation accuracy,0.661333,0.661333
6,Validation f1,0.656678,0.656678
7,Validation AUC,0.827453,0.827453


In [30]:
#записи чекпоінтів та данних для графіків
tb_callback = tf.keras.callbacks.TensorBoard(log_dir="logs/", histogram_freq=1)

checkpoint_path = "training/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath = checkpoint_path, save_weights_only=True, verbose=1)

In [32]:
#resulting model
metrics_list = ['accuracy', f1, AUC(name = 'AUC')]
the_model = CNN(299, 299, 3, 3, l2(0.00005), 'relu')
the_model.compile(optimizer = Adam(learning_rate=0.001), loss = 'categorical_crossentropy', metrics = metrics_list)
history = the_model.fit(train_generator, validation_data = validation_generator, epochs = 15, verbose = 1, callbacks = [tb_callback, cp_callback])
the_model.save(f'the_model')

Epoch 1/15
36/36 [==============================] - ETA: 0s - loss: 2.0583 - accuracy: 0.4467 - f1: 0.4394 - AUC: 0.6189
Epoch 1: saving model to training\cp-0001.ckpt
36/36 [==============================] - 100s 3s/step - loss: 2.0583 - accuracy: 0.4467 - f1: 0.4394 - AUC: 0.6189 - val_loss: 1.6624 - val_accuracy: 0.4387 - val_f1: 0.4073 - val_AUC: 0.6174
Epoch 2/15
36/36 [==============================] - ETA: 0s - loss: 1.2237 - accuracy: 0.5396 - f1: 0.5201 - AUC: 0.7203
Epoch 2: saving model to training\cp-0002.ckpt
36/36 [==============================] - 98s 3s/step - loss: 1.2237 - accuracy: 0.5396 - f1: 0.5201 - AUC: 0.7203 - val_loss: 1.2263 - val_accuracy: 0.5160 - val_f1: 0.5034 - val_AUC: 0.6961
Epoch 3/15
36/36 [==============================] - ETA: 0s - loss: 1.1429 - accuracy: 0.5591 - f1: 0.5381 - AUC: 0.7378
Epoch 3: saving model to training\cp-0003.ckpt
36/36 [==============================] - 97s 3s/step - loss: 1.1429 - accuracy: 0.5591 - f1: 0.5381 - AUC: 0.7378

INFO:tensorflow:Assets written to: the_model\assets


INFO:tensorflow:Assets written to: the_model\assets


In [33]:
%load_ext tensorboard
%tensorboard --logdir './logs' --host localhost